In [26]:
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

In [27]:
uri = "bolt://localhost:7687" 
user = "neo4j"  
password = "12345678"
driver = GraphDatabase.driver(uri=uri,auth=(user,password),database='neo4j')
session = driver.session(database='neo4j')

### Visualize the graph

In [28]:

result = session.run("CALL db.schema.visualization()")
w = GraphWidget(graph = result.graph())
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

### 1: Graph creation

In [29]:
session.run('''
CALL gds.graph.project(
  'ratings',
  ['User','Movie'],
  {
    RATED: {
      orientation: 'UNDIRECTED',
      properties: 'rating'
    }
  }
)
''')

### 2: Node embedding

In [30]:
session.run('''
CALL gds.fastRP.mutate('ratings',
  {
    embeddingDimension: 256,
    randomSeed: 7474,
    mutateProperty: 'embedding',
    relationshipWeightProperty: 'rating',
    iterationWeights: [0.8, 1, 1, 1]
  }
)
YIELD nodePropertiesWritten
''')

### 3: Node similarity with KNN

In [31]:
result = session.run('''
    CALL gds.knn.write('ratings', {
    topK: 5,
    nodeProperties: ['embedding'],
    randomSeed: 42,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0,
    writeRelationshipType: "SIMILAR",
    writeProperty: "score"
})
YIELD nodesCompared, relationshipsWritten, similarityDistribution
RETURN nodesCompared, relationshipsWritten, similarityDistribution.mean as meanSimilarity
''').to_df()
print(result)

   nodesCompared  relationshipsWritten  meanSimilarity
0           9808                 49040        0.327103


### 4: Explore the graph. Print similar users

In [32]:
similar_user = session.run('''
    MATCH (n:User)-[r:SIMILAR]->(m:User)
    RETURN n.id as person1, m.id as person2, r.score as similarity
    ORDER BY similarity DESCENDING, person1, person2 limit 10
''').to_df()
print(similar_user)

  person1 person2  similarity
0  user46  user56    0.970529
1  user56  user46    0.970529
2  user38   user8    0.970025
3   user8  user38    0.970025
4  user26  user56    0.969324
5  user56  user26    0.969324
6  user26  user37    0.964899
7  user37  user26    0.964899
8  user38  user58    0.963291
9  user58  user38    0.963291


### 5: Make moive recommendations by user KNN

In [38]:
query = '''
MATCH (u:User {id: 'user5'})
MATCH (u)-[r1:SIMILAR]->(similarUser)-[r2:RATED]->(m:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(m))
RETURN u.id AS userId, m.title AS movieTitle, m.id AS movieId, COLLECT(DISTINCT similarUser.id) AS similarUserIds
LIMIT 10
'''
result = session.run(query)
print(result.to_df())

  userId                          movieTitle movieId similarUserIds
0  user5                             Jumanji       2        [user8]
1  user5                           GoldenEye      10        [user8]
2  user5             American President, The      11        [user8]
3  user5  Twelve Monkeys (a.k.a. 12 Monkeys)      32        [user8]
4  user5                Seven (a.k.a. Se7en)      47        [user8]
5  user5                       Birdcage, The     141        [user8]
6  user5                            Net, The     185        [user8]
7  user5                         Nine Months     186        [user8]
8  user5                          Waterworld     208        [user8]
9  user5     Dumb & Dumber (Dumb and Dumber)     231        [user8]


#### visualize the graph

In [40]:
query = '''
MATCH (u:User {id: 'user5'})
MATCH (u)-[r1:SIMILAR]->(similarUser)-[r2:RATED]->(m:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(m))
RETURN u, similarUser, m, r1, r2
LIMIT 10
'''
result = session.run(query)
widget = GraphWidget(graph = result.graph())
widget.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

### 6: Recommendation by movie KNN

In [45]:
query = '''
MATCH (targetMovie:Movie {id: '68954'})
MATCH (targetMovie)-[similarity:SIMILAR]-(similarMovie:Movie)
WHERE similarMovie.id <> targetMovie.id
RETURN DISTINCT similarMovie.id AS MovieId, similarMovie.title AS movieTitle , similarity.score AS score
ORDER BY similarity.score DESC
LIMIT 10
'''
result = session.run(query)
print(result.to_df())

  MovieId                                      movieTitle     score
0   91529                          Dark Knight Rises, The  0.984773
1   79132                                       Inception  0.971491
2   58559                                Dark Knight, The  0.969354
3  112852                         Guardians of the Galaxy  0.967343
4    7153  Lord of the Rings: The Return of the King, The  0.965962
5   60069                                          WALL·E  0.963464
6   58826                    Shepherd: Border Patrol, The  0.000000


#### Visualize the graph

In [46]:
query = '''
MATCH (targetMovie:Movie {id: '68954'})
MATCH (targetMovie)-[similarity:SIMILAR]-(similarMovie:Movie)
WHERE similarMovie.id <> targetMovie.id
RETURN DISTINCT targetMovie, similarMovie, similarity
ORDER BY similarity.score DESC
LIMIT 10
'''
result = session.run(query)
widget = GraphWidget(graph = result.graph())
widget.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

#### 7: Recommendation by KNN + PageRank Algorithm

In [47]:
query = '''
CALL gds.graph.project(
  	'pagerank_collab',
  	['Movie', 'Movie'],
  	'SIMILAR',
  {
   	 relationshipProperties: {
      	score: {
        	property: 'score',
        	defaultValue: 0.0
      }
    }
  }
);
'''
result = session.run(query)


In [50]:
query = '''
    MATCH (src1:Movie {id: $movieid})
    CALL gds.pageRank.stream('pagerank_collab', {
    maxIterations: 20,
    dampingFactor: 0.85,
    sourceNodes: [src1],
    relationshipTypes: ['SIMILAR'],
    relationshipWeightProperty: 'score'
    })
    YIELD nodeId, score
    WHERE gds.util.asNode(nodeId).id <> src1.id
    WITH gds.util.asNode(nodeId) AS movie, score
    RETURN movie.id AS movieId, movie.title AS movieTitle, score
    ORDER BY score DESC
    LIMIT $n;
'''
result = session.run(query, movieid='68954', n=10)
print(result.to_df())

  movieId                                      movieTitle     score
0   58559                                Dark Knight, The  0.076272
1   79132                                       Inception  0.069243
2    7153  Lord of the Rings: The Return of the King, The  0.068999
3   91529                          Dark Knight Rises, The  0.057266
4  112852                         Guardians of the Galaxy  0.046662
5   60069                                          WALL·E  0.028320
6  122900                                         Ant-Man  0.023766
7   89745                                   Avengers, The  0.023307
8   73017                                 Sherlock Holmes  0.019971
9   33794                                   Batman Begins  0.017938
